In [2]:
import pandas as pd

f_race = "data/race-IL-ziptab-2013-2017/ACS_17_5YR_B02001/ACS_17_5YR_B02001_with_ann.csv"

fr = pd.read_csv(f_race)

fr = fr.drop(0)
fr.head()

# fr['index'] = 

,GEO.id,GEO.id2,GEO.display-label,HD01_VD01,HD02_VD01,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,...,HD01_VD06,HD02_VD06,HD01_VD07,HD02_VD07,HD01_VD08,HD02_VD08,HD01_VD09,HD02_VD09,HD01_VD10,HD02_VD10
1,8600000US60002,60002,ZCTA5 60002,24217,322,22261,481,540,280,0,...,0,20,460,205,619,242,24,36,595,245
2,8600000US60004,60004,ZCTA5 60004,50941,871,43436,1028,508,167,29,...,26,31,1532,521,913,281,130,124,783,263
3,8600000US60005,60005,ZCTA5 60005,30148,977,25015,1101,843,394,53,...,0,23,942,434,630,269,226,198,404,194
4,8600000US60007,60007,ZCTA5 60007,33691,191,27540,836,338,151,20,...,30,48,1182,481,856,298,276,222,580,236
5,8600000US60008,60008,ZCTA5 60008,21930,566,15339,998,978,414,93,...,118,154,3245,898,363,204,16,17,347,202


In [3]:
chicago_zip_codes = [60290, 60601, 60602, 60603, 60604, 60605, 60606, 60607, 60608, 60610, 60611, 60614, 60615, 60618, 60619, 60622, 60623, 60624, 60628, 60609, 60612, 60613, 60616, 60617, 60620, 60621, 60625, 60626, 60629, 60630, 60632, 60636, 60637, 60631, 60633, 60634, 60638, 60641, 60642, 60643, 60646, 60647, 60652, 60653, 60656, 60660, 60661, 60664, 60639, 60640, 60644, 60645, 60649, 60651, 60654, 60655, 60657, 60659, 60666, 60668, 60673, 60677, 60669, 60670, 60674, 60675, 60678, 60680, 60681, 60682, 60686, 60687, 60688, 60689, 60694, 60695, 60697, 60699, 60684, 60685, 60690, 60691, 60693, 60696, 60701]
chicago_fr = fr.loc[fr['GEO.id2'].astype('int32').isin(chicago_zip_codes)]

race_ests = chicago_fr

race_ests.index = race_ests['GEO.id2']
race_ests = race_ests[['HD01_VD01', 'HD01_VD02', 'HD01_VD03', 'HD01_VD04', 'HD01_VD05', 'HD01_VD06', 'HD01_VD07', 'HD01_VD08']]
race_ests = race_ests.rename(index=str, columns={"HD01_VD01": "Total",
                                                 "HD01_VD02": "White",
                                                 "HD01_VD03": "Afr. Am.",
                                                 "HD01_VD04": "Am./AK Native",
                                                 "HD01_VD05": "Asian",
                                                 "HD01_VD06": "Native HI/Pac. Isl.",
                                                 "HD01_VD07": "Other race alone",
                                                 "HD01_VD08": "2+ races",})
race_ests = race_ests.astype('float64')

race_ests.head()

,Total,White,Afr. Am.,Am./AK Native,Asian,Native HI/Pac. Isl.,Other race alone,2+ races
GEO.id2,,,,,,,,
60601,13887.0,10384.0,1148.0,68.0,2140.0,0.0,61.0,86.0
60602,1277.0,976.0,11.0,57.0,232.0,0.0,0.0,1.0
60603,1197.0,706.0,27.0,0.0,413.0,0.0,0.0,51.0
60604,668.0,418.0,42.0,0.0,208.0,0.0,0.0,0.0
60605,26188.0,15813.0,4631.0,24.0,4420.0,10.0,239.0,1051.0


In [4]:
def simpson_index(total, cols):
    s = sum([col*(col-1.0) for col in cols])
    return 1.0 - (s/(total * (total - 1.0)))

race_ests["Simpson Index"] = race_ests.apply(lambda row: simpson_index(row["Total"], [row["White"], row["Afr. Am."],
                                                             row["Am./AK Native"], row["Asian"],
                                                             row["Native HI/Pac. Isl."], row["Other race alone"],
                                                             row["2+ races"]]), axis=1)

race_ests.head()

,Total,White,Afr. Am.,Am./AK Native,Asian,Native HI/Pac. Isl.,Other race alone,2+ races,Simpson Index
GEO.id2,,,,,,,,,
60601,13887.0,10384.0,1148.0,68.0,2140.0,0.0,61.0,86.0,0.410237
60602,1277.0,976.0,11.0,57.0,232.0,0.0,0.0,1.0,0.381084
60603,1197.0,706.0,27.0,0.0,413.0,0.0,0.0,51.0,0.531201
60604,668.0,418.0,42.0,0.0,208.0,0.0,0.0,0.0,0.508291
60605,26188.0,15813.0,4631.0,24.0,4420.0,10.0,239.0,1051.0,0.573963


In [5]:
def grade(z_score):
    if z_score >= 1.96:
        return 'A+'
    elif z_score >= 1.28:
        return 'A'
    elif z_score >= 0.84:
        return 'A-'
    elif z_score >= 0.44:
        return 'B+'
    elif z_score >= 0:
        return 'B'
    elif z_score >= -0.44:
        return 'B-'
    elif z_score >= -0.84:
        return 'C+'
    elif z_score >= -1.28:
        return 'C'
    elif z_score >= -1.96:
        return 'C-'
    elif z_score >= -2.25:
        return 'D+'
    elif z_score >= -2.25:
        return 'D'
    elif z_score >= -2.50:
        return 'D-'
    return 'F'

race_ests['z_score'] = (race_ests["Simpson Index"] - race_ests["Simpson Index"].mean()) / race_ests["Simpson Index"].std()
race_ests['grade'] = race_ests['z_score'].apply(grade)

race_ests


,Total,White,Afr. Am.,Am./AK Native,Asian,Native HI/Pac. Isl.,Other race alone,2+ races,Simpson Index,z_score,grade
GEO.id2,,,,,,,,,,,
60601,13887.0,10384.0,1148.0,68.0,2140.0,0.0,61.0,86.0,0.410237,-0.044668,B-
60602,1277.0,976.0,11.0,57.0,232.0,0.0,0.0,1.0,0.381084,-0.205522,B-
60603,1197.0,706.0,27.0,0.0,413.0,0.0,0.0,51.0,0.531201,0.622752,B+
60604,668.0,418.0,42.0,0.0,208.0,0.0,0.0,0.0,0.508291,0.496343,B+
60605,26188.0,15813.0,4631.0,24.0,4420.0,10.0,239.0,1051.0,0.573963,0.858691,A-
60606,2959.0,2235.0,76.0,0.0,432.0,0.0,54.0,162.0,0.404319,-0.077320,B-
60607,28928.0,17396.0,3681.0,30.0,6399.0,26.0,377.0,1019.0,0.571856,0.847066,A-
60608,78877.0,37203.0,14147.0,529.0,8712.0,0.0,16759.0,1527.0,0.687616,1.485773,A
60609,60994.0,29002.0,14628.0,402.0,3755.0,12.0,11885.0,1310.0,0.674141,1.411422,A


In [6]:
race_ests.to_csv('racial_diversity.csv')